In [ ]:
%cd mmdetectionㅇㅇ

In [ ]:

import os
import json


from mmdet.apis import DetInferencer
from mmengine.config import Config


device = 'cuda:0'
config_path = ""
checkpoint = ""
cfg = Config.fromfile(config_path)

image_path_prefix = "../datasets/extra_data_1/train/"
save_prefix = "../datasets/anno/"

extra_image_num = 500
pseudo_thres = 0.65

In [ ]:
inferencer = DetInferencer(cfg, checkpoint, device)


In [ ]:

with open('../datasets/extra_data_1/train/_annotations.coco.json', 'r') as file:
    ext_train_data = json.load(file)

In [ ]:
total_num=0
ret_list=[]
for idx,info in enumerate(ext_train_data["annotations"]):
    file_name = ext_train_data["images"][info["image_id"]]["file_name"]
    image_path = os.path.join(image_path_prefix,file_name)
    results = inferencer(image_path)

    if total_num>extra_image_num:
        break
    total_num+=1
    print(f"total_num : {total_num}")
    for ret_idx,bbox in enumerate(results["predictions"][0]["bboxes"]):
        if results['predictions'][0]["scores"][ret_idx] > pseudo_thres:
            min_x, min_y, max_x, max_y = bbox
            ret = {
                    "image_id" : file_name,
                    "category_id" : results['predictions'][0]["labels"][ret_idx],
                    "bbox":[
                        min_x,
                        min_y,
                        max_x-min_x,
                        max_y-min_y
                    ],
                    "score":results['predictions'][0]["scores"][ret_idx],
                }
            ret_list.append(ret)




In [ ]:
with open(os.path.join(save_prefix,"extra_pseudo_ann.txt"), "w") as file:
    json.dump(ret_list, file, indent=4)